In [1]:
# 1. Setup & Configuration

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.auto import tqdm
import gc

# Configuration - should match the training notebook
class CFG:
    data_dir = '.'
    test_path = os.path.join(data_dir, 'test')
    sample_submission_path = os.path.join(data_dir, 'sample_submission.csv')
    model_paths = [f'tf_efficientnet_b0_ns_fold{i}_best.pth' for i in range(5)]
    
    model_name = 'tf_efficientnet_b0_ns'
    img_size = 256
    in_channels = 3
    num_classes = 1
    
    batch_size = 128 # Can be larger for inference
    n_folds = 5
    
    tta = True # Enable Test-Time Augmentation
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device: {CFG.device}")
print(f"TTA enabled: {CFG.tta}")
print(f"Models to load: {CFG.model_paths}")

Using device: cuda
TTA enabled: True
Models to load: ['tf_efficientnet_b0_ns_fold0_best.pth', 'tf_efficientnet_b0_ns_fold1_best.pth', 'tf_efficientnet_b0_ns_fold2_best.pth', 'tf_efficientnet_b0_ns_fold3_best.pth', 'tf_efficientnet_b0_ns_fold4_best.pth']


In [2]:
# 2. Data Loading & Preprocessing

from utils_preproc import load_and_preprocess # Import shared preprocessing

def get_test_file_path(image_id):
    return f"{CFG.test_path}/{image_id[0]}/{image_id}.npy"

test_df = pd.read_csv(CFG.sample_submission_path)
test_df['file_path'] = test_df['id'].apply(get_test_file_path)

print("Test dataframe:")
print(test_df.head())

# TTA transforms - only no-flip and h-flip, as per expert advice
def get_transforms(h_flip=False):
    transforms = [A.Resize(CFG.img_size, CFG.img_size)]
    if h_flip:
        transforms.append(A.HorizontalFlip(p=1.0))
    transforms.append(ToTensorV2())
    return A.Compose(transforms)

class SETITestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_paths = df['file_path'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        
        # Use the centralized preprocessing function to ensure consistency
        image = load_and_preprocess(file_path, do_asinh=True)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        return image

Test dataframe:
                id  target                     file_path
0  0cee567456cd304     0.5  ./test/0/0cee567456cd304.npy
1  5451b45281c65a7     0.5  ./test/5/5451b45281c65a7.npy
2  f8cc6cea820282d     0.5  ./test/f/f8cc6cea820282d.npy
3  25e21ba81a64742     0.5  ./test/2/25e21ba81a64742.npy
4  aafa910406b1db2     0.5  ./test/a/aafa910406b1db2.npy


In [3]:
# 3. Model Definition

class SETIModel(nn.Module):
    def __init__(self, model_name=CFG.model_name, pretrained=False): # Set pretrained=False for loading local weights
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=CFG.in_channels, num_classes=CFG.num_classes)

    def forward(self, x):
        x = self.model(x)
        return x

In [4]:
# 4. Inference Function

def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    
    pbar = tqdm(test_loader, desc='Inference')
    with torch.no_grad():
        for images in pbar:
            images = images.to(device)
            
            # Use AMP for inference as well for consistency and speed
            with torch.cuda.amp.autocast():
                y_preds = model(images)
            
            predictions.append(y_preds.sigmoid().to('cpu').numpy())
            
    return np.concatenate(predictions).flatten()

In [5]:
# 5. Main Inference & Submission Generation

all_preds = []

for model_path in CFG.model_paths:
    print(f"--- Inferring with {model_path} ---")
    model = SETIModel()
    model.load_state_dict(torch.load(model_path))
    
    fold_preds = []
    
    if CFG.tta:
        # TTA: no flip, h-flip (as per expert advice and training setup)
        tta_transforms = [
            get_transforms(h_flip=False),
            get_transforms(h_flip=True),
        ]
        
        tta_preds = []
        for i, transform in enumerate(tta_transforms):
            print(f"  > TTA pass {i+1}/{len(tta_transforms)}")
            test_dataset = SETITestDataset(test_df, transform=transform)
            test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=4, pin_memory=True)
            
            preds = inference(model, test_loader, CFG.device)
            tta_preds.append(preds)
            
        # Average TTA predictions
        fold_preds = np.mean(tta_preds, axis=0)
        
    else:
        # No TTA
        test_dataset = SETITestDataset(test_df, transform=get_transforms())
        test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=4, pin_memory=True)
        fold_preds = inference(model, test_loader, CFG.device)
        
    all_preds.append(fold_preds)
    
    del model
    gc.collect()
    torch.cuda.empty_cache()

# Ensemble by averaging predictions across folds
final_preds = np.mean(all_preds, axis=0)

# Create submission file
submission = pd.DataFrame({'id': test_df['id'], 'target': final_preds})
submission.to_csv('submission.csv', index=False)

print("\nSubmission file created: submission.csv")
print(submission.head())

/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


--- Inferring with tf_efficientnet_b0_ns_fold0_best.pth ---
  > TTA pass 1/2


Inference:   0%|          | 0/47 [00:00<?, ?it/s]

/tmp/ipykernel_19643/4095927241.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Inference:   2%|▏         | 1/47 [00:01<01:01,  1.35s/it]

Inference:   6%|▋         | 3/47 [00:01<00:17,  2.59it/s]

...

Inference:  87%|████████▋ | 41/47 [00:09<00:01,  4.44it/s]

Inference:  91%|█████████▏| 43/47 [00:09<00:00,  5.67it/s]

Inference:  96%|█████████▌| 45/47 [00:10<00:00,  4.50it/s]

Inference: 100%|██████████| 47/47 [00:10<00:00,  4.44it/s]


Submission file created: submission.csv
                id    target
0  0cee567456cd304  0.079834
1  5451b45281c65a7  0.229614
2  f8cc6cea820282d  0.112366
3  25e21ba81a64742  0.179443
4  aafa910406b1db2  0.157227
